In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
Path='/content/drive/MyDrive/Project'

In [4]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=50

In [5]:
import tensorflow as tf

IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    Path,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,

)



Found 2192 files belonging to 3 classes.


In [6]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [7]:
len(dataset)

69

In [8]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1

    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [9]:
train_ds,val_ds,test_ds=get_dataset_partitions_tf(dataset)

In [10]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [11]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(256, 256, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,839,363 (56.61 MB)

 Trainable params: 14,839,363 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [14]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=10,
)


Epoch 1/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 403s 5s/step - accuracy: 0.6468 - loss: 1.4635 - val_accuracy: 0.9375 - val_loss: 0.2256
Epoch 2/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 258s 5s/step - accuracy: 0.9249 - loss: 0.2037 - val_accuracy: 0.9635 - val_loss: 0.1274
Epoch 3/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 249s 5s/step - accuracy: 0.9554 - loss: 0.1331 - val_accuracy: 0.9844 - val_loss: 0.0671
Epoch 4/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 277s 5s/step - accuracy: 0.9673 - loss: 0.0961 - val_accuracy: 0.9635 - val_loss: 0.0836
Epoch 5/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 321s 5s/step - accuracy: 0.9833 - loss: 0.0494 - val_accuracy: 0.9792 - val_loss: 0.0547
Epoch 6/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 263s 5s/step - accuracy: 0.9850 - loss: 0.0464 - val_accuracy: 0.9844 - val_loss: 0.0409
Epoch 7/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 264s 5s/step - accuracy: 0.9778 - loss: 0.0577 - val_accuracy: 0.9844 - val_loss: 0.0584
Epoch 8/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 256s 5s/step - accuracy: 0.9987 - loss: 0.0115 - val_accuracy: 0.9844 - v

In [15]:
model.save('classify_leaf.keras')

In [16]:
loss, accuracy = model.evaluate(val_ds)
print(f'Validation accuracy: {accuracy:.2f}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.9891 - loss: 0.0415
Validation accuracy: 0.98


In [23]:

class_names = ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']


predictions = new_model.predict(img_array)


predicted_class = class_names[np.argmax(predictions)]
print(f'The image is classified as: {predicted_class}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
The image is classified as: Potato___Late_blight
